In [2]:
!pip install torch transformers -q


In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("fill-mask", model="InstaDeepAI/nucleotide-transformer-2.5b-multi-species")

AMD1_leader = "GCTTACACAGTATGGCCGGCGACATTAGCTAGCGCTCGCTCTACTCTCTCTAACGGGAAAGCAGCGGAATACAAGAGACTGAACTGTATCTGCCTCTATTTCCAAAAGACTCACGTTCAACTTTCGCTCACACAAAGCCGGGAAAATTTTATTAGTCCTTTTTTTAAAAAAAGTTAATATAAAATTATAGCAAAAAAAAAAAGGAACCTGAACTTTAGTAACACAGCTGGAACAATCCGCAGCGGCGGCGGCAGCGGCGGGAGAAGAGGTTTAATTTAGTTGATTTTCTGTGGTTGTTGGTTGTTCGCTAGTCTCACGGTGATGGAAGCTGCACATTTTTTCGAAGGGACCGAGAAGCTGCTGGAGGTTTGGTTCTCCCGGCAGCAGCCCGACGCAAACCAAGGATCTGGGGATCTTCGCACTATCCCAAG"

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/46.0k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.91G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/278M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/28.7k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/101 [00:00<?, ?B/s]

In [3]:
pipe(AMD1_leader.replace("A", "<mask>"))

[[{'score': 0.6416748762130737,
   'token': 4104,
   'token_str': 'N',
   'sequence': '<cls> G C T T N C <mask> C <mask> G T <mask> TGGCCG G C G <mask> C <mask> T T <mask> G C T <mask> GCGCTC G C T C T <mask> CTCTCT C T <mask> <mask> C G G G <mask> <mask> <mask> G C <mask> G C G G <mask> <mask> T <mask> C <mask> <mask> G <mask> G <mask> C T G <mask> <mask> C T G T <mask> TCTGCC T C T <mask> T T T C C <mask> <mask> <mask> <mask> G <mask> C T C <mask> C G T T C <mask> <mask> CTTTCG C T C <mask> C <mask> C <mask> <mask> <mask> GCCGGG <mask> <mask> <mask> <mask> T T T T <mask> T T <mask> GTCCTT T T T T T <mask> <mask> <mask> <mask> <mask> <mask> <mask> G T T <mask> <mask> T <mask> T <mask> <mask> <mask> <mask> T T <mask> T <mask> G C <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> G G <mask> <mask> C C T G <mask> <mask> C T T T <mask> G T <mask> <mask> C <mask> C <mask> G C T G G <mask> <mask> C <mask> <mask> T C C G C <mask> GCGGCG G C G G C <mask> GCGGCG G G 

In [4]:
import torch
from torch import nn
from transformers import AutoModelForTokenClassification, AutoTokenizer, Trainer, TrainingArguments


In [5]:
# Load pre-trained model and tokenizer
model_name = "InstaDeepAI/nucleotide-transformer-2.5b-multi-species"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=2)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of EsmForTokenClassification were not initialized from the model checkpoint at InstaDeepAI/nucleotide-transformer-2.5b-multi-species and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
import torch
from transformers import AutoTokenizer

# Make sure to initialize your tokenizer beforehand
tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t30_150M_UR50D")  # Replace with your model

class RNADataset(torch.utils.data.Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq = self.sequences[idx]
        label = self.labels[idx]

        # Tokenize the sequence
        encoded = tokenizer(seq, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
        
        # Ensure that labels are a tensor with the correct shape
        encoded["labels"] = torch.tensor(label, dtype=torch.float)  # or dtype=torch.long for integer labels
        
        # Squeeze the tensors to remove the extra batch dimension (size 1)
        for key in encoded.keys():
            encoded[key] = encoded[key].squeeze(0)  # Remove the batch dimension

        return encoded


tokenizer_config.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [8]:

# Load and preprocess your data
# Each sequence is a string of nucleotides, each label is a list of 0s and 1s (1 for translated)
train_sequences = ["AUGCUAAAG", "GCAUAAAGCU"]
train_labels = [[1,1,1,0,0,0,1,1,1], [0,0,0,1,1,1,0,0,0]]
train_dataset = RNADataset(train_sequences, train_labels)

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=1,
    weight_decay=0.01,
    logging_dir="./logs",
)

# Create Trainer and fine-tune the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

trainer.train()

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_rna_region_model")



Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


ValueError: Expected input batch_size (512) to match target batch_size (9).

In [9]:
class RNADataset(torch.utils.data.Dataset):
    def __init__(self, sequences, labels, tokenizer):
        self.sequences = sequences
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq = self.sequences[idx]
        label = self.labels[idx]
        
        # Tokenize the sequence
        encoding = self.tokenizer(seq, 
                                  truncation=True, 
                                  padding='max_length', 
                                  max_length=len(seq),  # Use sequence length as max_length
                                  return_tensors='pt')
        
        # Remove the batch dimension added by the tokenizer
        input_ids = encoding['input_ids'].squeeze(0)
        attention_mask = encoding['attention_mask'].squeeze(0)
        
        # Convert labels to tensor
        labels_tensor = torch.tensor(label)
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels_tensor
        }

# Load and preprocess your data
train_sequences = ["AUGCUAAAG", "GCAUAAAGCU"]
train_labels = [[1,1,1,0,0,0,1,1,1], [0,0,0,1,1,1,0,0,0,0]]
train_dataset = RNADataset(train_sequences, train_labels, tokenizer)

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=1,
    save_steps=5,
    evaluation_strategy="steps",
    eval_steps=5,
    load_best_model_at_end=True,
)

# Create Trainer and fine-tune the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

trainer.train()

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_rna_region_model")


/data1/jack/miniconda3/envs/TranslonFormer/lib/python3.12/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss,Validation Loss


Sequence: AUGCUAAAG
Predicted translated regions: [0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
# Function to predict translated regions
def predict_translated_regions(sequence):
    inputs = tokenizer(sequence, return_tensors="pt", padding=True, truncation=True, max_length=len(sequence))
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    return predictions[0].tolist()  # Convert to list for easier interpretation

# Example usage
test_sequence = "AUGCUAAAG"
predicted_regions = predict_translated_regions(test_sequence)
print(f"Sequence: {test_sequence}")
print(f"Predicted translated regions: {predicted_regions}")